In [163]:
import pandas as pd
data = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data\_train.json',lines=True)

In [164]:
import pandas as pd

In [165]:
df = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data\_train.json',lines=True,orient='records')
df = df[pd.notnull(df['label'])]

In [166]:
print(df.head(11))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    text  \
0                                                           

In [167]:
with open('C:\study\研一上\作业\自然语言处理\data\data\label_list.txt', 'r') as f:
    lines = f.readlines()
    labels = []
    for line in lines:
        labels.append(line.strip())
    f.close()

In [168]:
labels

['Contact',
 'Contact_Address_Book',
 'Contact_City',
 'Contact_E_Mail_Address',
 'Contact_Password',
 'Contact_Phone_Number',
 'Contact_Postal_Address',
 'Contact_ZIP',
 'Demographic',
 'Demographic_Age',
 'Demographic_Gender',
 'Facebook_SSO',
 'Identifier',
 'Identifier_Ad_ID',
 'Identifier_Cookie_or_similar_Tech',
 'Identifier_Device_ID',
 'Identifier_IMEI',
 'Identifier_IMSI',
 'Identifier_IP_Address',
 'Identifier_MAC',
 'Identifier_Mobile_Carrier',
 'Identifier_SIM_Serial',
 'Identifier_SSID_BSSID',
 'Location',
 'Location_Bluetooth',
 'Location_Cell_Tower',
 'Location_GPS',
 'Location_IP_Address',
 'Location_WiFi',
 'SSO',
 'No_Mentioned']

In [190]:
import re

In [191]:
REPLACE_BY_SPACE_RE = re.compile('[/{}\[\]\|@,;]')#括号直接去掉，而不是分成两个词
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')# 不显示数字了

In [192]:
def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text  # HTML decoding
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    # text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # delete stopwors from text
    return text

df['text'] = df['text'].apply(clean_text)

In [174]:
df.iloc[0]['text']

'privacy policy this privacy policy hereafter referred to as the privacy policy is applicable to our websites  apps and to all games and other activitieshereafter referred to as the our products that are offered by us on or through our products tiny piece  having its registered office at ajeltake road  ajeltake island  majuro  republic of the marshall island mh  hereafter referred to as g g may use affiliates or reputable third parties services for the processing of personal data collected on or through our products by using or accessing our products  you are accepting the practices described in this privacy policy'

In [175]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

In [176]:
labels = df['label'].values
labels_dic = {}
for label_list in labels:
    for label in label_list:
        if label not in labels_dic:
            labels_dic[label] = 1
        else:
            labels_dic[label] += 1
labels_dic

{'No_Mentioned': 7332,
 'Identifier_Cookie_or_similar_Tech': 1001,
 'Identifier_IP_Address': 628,
 'Demographic': 146,
 'Demographic_Age': 188,
 'Contact_E_Mail_Address': 882,
 'Contact_Phone_Number': 441,
 'Contact_Postal_Address': 318,
 'Facebook_SSO': 134,
 'SSO': 189,
 'Contact_Address_Book': 159,
 'Contact_Password': 166,
 'Identifier_Ad_ID': 166,
 'Identifier_Device_ID': 472,
 'Identifier_IMEI': 75,
 'Identifier_IMSI': 11,
 'Location': 723,
 'Identifier_SSID_BSSID': 9,
 'Location_Cell_Tower': 130,
 'Location_IP_Address': 144,
 'Identifier_MAC': 121,
 'Demographic_Gender': 153,
 'Contact': 164,
 'Identifier': 104,
 'Identifier_Mobile_Carrier': 89,
 'Contact_ZIP': 66,
 'Location_Bluetooth': 136,
 'Location_GPS': 215,
 'Location_WiFi': 186,
 'Identifier_SIM_Serial': 21,
 'Contact_City': 48}

In [177]:
X_train = list(df['text'])
X_train

['privacy policy this privacy policy hereafter referred to as the privacy policy is applicable to our websites  apps and to all games and other activitieshereafter referred to as the our products that are offered by us on or through our products tiny piece  having its registered office at ajeltake road  ajeltake island  majuro  republic of the marshall island mh  hereafter referred to as g g may use affiliates or reputable third parties services for the processing of personal data collected on or through our products by using or accessing our products  you are accepting the practices described in this privacy policy',
 ' about our products  our products offer a diverse  current  and exciting mix of games created by g  as well as games created by independent developers and g partners players can access our products to play games without registering  however  they may choose to register to create a public or semipublic profile and to save games  high scores  and comments on this profile 

In [178]:
y_train = df['label']

In [179]:
y_train 

0                                                                                                             [No_Mentioned]
1                                                                                                             [No_Mentioned]
2                                                                 [Identifier_Cookie_or_similar_Tech, Identifier_IP_Address]
3                                                                                        [Identifier_Cookie_or_similar_Tech]
4                                                                                                             [No_Mentioned]
                                                                ...                                                         
10145                                                                                                         [No_Mentioned]
10146                              [Contact_City, Contact_E_Mail_Address, Demographic_Age, Demographic_Gender, Facebook_SSO]


In [180]:
y_train = list(y_train)

In [181]:
vf = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data2\_valid.json',lines=True)
vf = vf[pd.notnull(vf['label'])]

In [182]:
print(vf.head(5))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              text  \
0                                                                                                 

In [183]:
X_val = list(vf['text'])
X_val

['Skip to main content Home',
 'Thank you for using our services. This privacy policy went into effect on January 1, 2017.',
 'The Weather Company, an IBM Business, (sometimes, "we" or "us") is committed to protecting your privacy. This Privacy Policy is important and we encourage you to carefully read it.',
 'This Privacy Policy discloses how we collect, use and share information that we gather about you on the weather.com® website (the "Site") and our software applications, mobile applications and other websites and services that reference or link to this Privacy Policy (together, the "Services"). This Privacy Policy also describes the choices you have regarding our use of and your ability to correct the information. The Services do not include websites, mobile applications or other services that link to another privacy policy. Our TRUSTe Certification How We Collect and Use Personal Data and Other Information Data Collection Technologies and Consumer Choice How We Share or Disclose 

In [184]:
y_val = vf['label']
y_val

0       [No_Mentioned]
1       [No_Mentioned]
2       [No_Mentioned]
3       [No_Mentioned]
4       [No_Mentioned]
             ...      
2812    [No_Mentioned]
2813    [No_Mentioned]
2814    [No_Mentioned]
2815    [No_Mentioned]
2816    [No_Mentioned]
Name: label, Length: 2817, dtype: object

In [185]:
y_val = list(y_val)

In [186]:
CV = CountVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
feature = CV.fit_transform(X_train)
print(feature)

  (0, 13669)	4
  (0, 13362)	4
  (0, 17786)	2
  (0, 14515)	3
  (0, 17987)	5
  (0, 2596)	3
  (0, 17018)	5
  (0, 8994)	1
  (0, 2253)	1
  (0, 12478)	5
  (0, 19824)	1
  (0, 2394)	1
  (0, 1373)	2
  (0, 1063)	1
  (0, 7257)	1
  (0, 12287)	1
  (0, 13835)	4
  (0, 16876)	1
  (0, 2442)	2
  (0, 11452)	1
  (0, 3545)	2
  (0, 18920)	1
  (0, 11498)	2
  (0, 11744)	4
  (0, 17892)	2
  :	:
  (10149, 3318)	1
  (10149, 4287)	1
  (10149, 8484)	1
  (10149, 18011)	1
  (10149, 8334)	1
  (10149, 10949)	1
  (10149, 9657)	1
  (10149, 10633)	1
  (10149, 3811)	1
  (10149, 10904)	1
  (10149, 8572)	1
  (10149, 13101)	1
  (10149, 11102)	1
  (10149, 3321)	1
  (10149, 1717)	1
  (10149, 1307)	1
  (10149, 8485)	1
  (10149, 12301)	1
  (10149, 1623)	1
  (10149, 7786)	1
  (10149, 15479)	1
  (10149, 7979)	1
  (10149, 4292)	1
  (10149, 16988)	1
  (10149, 16150)	1


In [187]:
tfidf = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
feature = tfidf.fit_transform(X_train)
print(feature)

  (0, 13669)	0.12054589234054167
  (0, 13362)	0.1345501507432452
  (0, 17786)	0.05762072203597924
  (0, 14515)	0.2314493390514342
  (0, 17987)	0.08749375973566371
  (0, 2596)	0.08904679756443848
  (0, 17018)	0.09128336994027778
  (0, 8994)	0.03020890570228268
  (0, 2253)	0.05236847057066129
  (0, 12478)	0.12819285151262622
  (0, 19824)	0.048459639383102666
  (0, 2394)	0.04964332585994351
  (0, 1373)	0.036752850642654517
  (0, 1063)	0.04268202447941398
  (0, 7257)	0.058809059443774646
  (0, 12287)	0.030330535156432546
  (0, 13835)	0.18509500353973787
  (0, 16876)	0.026861234065205276
  (0, 2442)	0.06529519209319604
  (0, 11452)	0.06983113024362772
  (0, 3545)	0.057690355961900266
  (0, 18920)	0.032053390094786346
  (0, 11498)	0.05982421641376364
  (0, 11744)	0.08417340466306471
  (0, 17892)	0.08759031985132741
  :	:
  (10149, 3318)	0.12453761631872266
  (10149, 4287)	0.11717697211995326
  (10149, 8484)	0.120430956499522
  (10149, 18011)	0.12997010626041947
  (10149, 8334)	0.105990143970

In [188]:
y_train

[['No_Mentioned'],
 ['No_Mentioned'],
 ['Identifier_Cookie_or_similar_Tech', 'Identifier_IP_Address'],
 ['Identifier_Cookie_or_similar_Tech'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['Identifier_Cookie_or_similar_Tech'],
 ['No_Mentioned'],
 ['Demographic', 'Demographic_Age'],
 ['Identifier_Cookie_or_similar_Tech'],
 ['Contact_E_Mail_Address', 'Demographic_Age'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address', 'Contact_Phone_Number', 'Contact_Postal_Addr

In [189]:
mlb = MultiLabelBinarizer(classes=sorted(labels_dic.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

In [ ]:
y_train

In [193]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def evaluation(y_val, predicted):
    accuracy = accuracy_score(y_val, predicted)
    f1_score_macro = f1_score(y_val, predicted, average='macro')
    print("accuracy:", accuracy)
    print('f1_score_macro',f1_score_macro)

In [203]:
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')),
    ('lsvc', OneVsRestClassifier(LinearSVC()))
])
pipeline1.fit(X_train,y_train)
predicted = pipeline1.predict(X_val)
evaluation(y_val, predicted)

accuracy: 0.8150514731984381
f1_score_macro 0.40990599435086034


In [195]:
test_df = pd.read_csv('ceshitest.txt', sep='\0', header=None)
test_df.columns = ['text']

In [196]:
test_df

,text
0,"More All All Live Stations Artists Songs Podcasts Share Privacy and Cookie Notice November 10, 2016 Versión en Español UPDATED AS OF: November 10, 2016 iHeartMedia, Inc. (""iHeartMedia"", ""we"", ""us"", or ""our"") is committed to maintaining your confidence and trust as it relates to the privacy of your information. Please read below and learn how we collect, protect, share and use your information as part of our technology platforms, including, without limitation, our websites, web pages, interactive features, applications, Twitter and Facebook pages, and Mobile Application (""Platforms"")."
1,"QUICK GUIDE TO CONTENTS INFORMATION COLLECTED ON OUR PLATFORMS HOW WE USE THE INFORMATION WE COLLECT SHARING OF INFORMATION INFORMATION WE RECEIVE FROM THIRD PARTIES YOUR PRIVACY RIGHTS, CHOICE AND ACCESS ADVERTISING/BEHAVIORAL TARGETING; HOW TO OPT-OUT CHILDREN SECURITY OF YOUR INFORMATION OTHER SITES CONSENT TO PROCESSING AND TRANSFER OF INFORMATION CHANGES CONTACT US"
2,"1. INFORMATION COLLECTED ON OUR PLATFORMS Information You Provide To Us We or our service providers may collect Personal Information (information that can be used to identify you as an individual) such as your name, email, telephone number, home address, demographic information (such as zip code, age and gender), location information, or payment information (such as account or credit card number). The types of Personal Information collected may vary depending on your use of the features of the Platforms. For example, your credit card number and other payment related information may be collected in connection with your purchase of products and/or services through the Platforms. We post customer testimonials/video testimonials on our Platforms which may contain personally identifiable information. If we want to post a customer's name along with their testimonial, we obtain the customer's consent via email prior to posting the testimonial."
3,"Information Collected Automatically Usage Information. Whenever you visit or interact with the Platforms, we, as well as any third-party advertisers and/or service providers, may use a variety of technologies that automatically or passively collect information about how the Platforms are accessed and used (""Usage Information""). Usage Information may include Device Identifiers (as defined below), browser type, device type, operating system, application version, the page served, the time, the preceding page views, and your use of features or applications on the Platforms, such as song history, interaction with friends and group activities. This information helps us keep our Platforms fresh and interesting to our visitors and allows us to tailor content to a visitor's interests."
4,"Device Identifier. We or our service providers automatically collect your IP address or other unique identifier (""Device Identifier"") for the Device (computer, mobile phone, tablet or other device) you use to access the Platforms. A Device Identifier is a number such as a mobile advertising identifier (Android AAID or Apple IDFA) that is assigned to your Device when you access a website or its servers, and our computers identify your Device by its Device Identifier. We may use a Device Identifier to, among other things, administer the Platforms, help diagnose problems with our servers, analyze trends, provide attribution metrics to our advertisers and partners, track users' web page movements, help identify you and your shopping cart, and gather broad demographic information for aggregate use."
...,...
2535,We retain the information we collect as described in this Privacy Policy for as long as necessary to fulfill our business and/or operational purposes or comply with a legal request. We may also choose to anonymize parts of the information we collect so that it can no longer be identifiable or attributable to any person if we want to keep it for longer periods of time.
2536,Transfer of Data Outside Your Territory.
2537,The Software & Ser

In [197]:
testSet = list(test_df['text'])

In [198]:
test_predicted_labels = pipeline1.predict(testSet)

In [199]:
test_predicted_labels

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [200]:
test_original_labels = mlb.inverse_transform(test_predicted_labels)

In [201]:
test_original_labels

[('No_Mentioned',),
 ('No_Mentioned',),
 ('Contact_E_Mail_Address', 'Contact_Phone_Number', 'Contact_Postal_Address'),
 (),
 ('Identifier_Device_ID', 'Identifier_IP_Address'),
 ('Location', 'Location_Bluetooth', 'Location_GPS', 'Location_WiFi'),
 ('Identifier_Cookie_or_similar_Tech',),
 ('Identifier_Cookie_or_similar_Tech',),
 ('Identifier_Cookie_or_similar_Tech',),
 ('Location',),
 ('Identifier_Cookie_or_similar_Tech',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('Contact_E_Mail_Address', 'SSO'),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('Location', 'Location_Bluetooth', 'Location_GPS', 'Location_WiFi'),
 ('Identifier_Cookie_or_similar_Tech',),
 (),
 ('No_Mentioned',),
 ('No_Mentioned',),
 (),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 (),
 (),
 ('No_M

In [202]:
with open("test_labels.txt", "w") as file:
    for tol in test_original_labels:
        line = ','.join(tol)
        if not line.strip():
            line = "No_Mentioned"
        file.write(line + '\n')

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, hamming_loss

# 读取数据
df = pd.read_json(r'C:\study\研一上\作业\自然语言处理\data\data\_train.json', lines=True, orient='records')
df = df[pd.notnull(df['label'])]

# 清洗文本的函数
REPLACE_BY_SPACE_RE = re.compile('[/{}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z\s#+_]')  # 修改了这里
def clean_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    return text

df['text'] = df['text'].apply(clean_text)

# 读取标签
with open(r'C:\study\研一上\作业\自然语言处理\data\data\label_list.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# 数据预处理
labels_dic = {label: 1 for label in set(label for sublist in df['label'] for label in sublist)}

# 训练测试集划分
X_train = list(df['text'])
y_train = df['label']
vf = pd.read_json(r'C:\study\研一上\作业\自然语言处理\data\data2\_valid.json', lines=True)  # 添加了闭合的括号
vf = vf[pd.notnull(vf['label'])]
X_val = list(vf['text'])
y_val = vf['label']

# 特征提取
tfidf = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,3), token_pattern='(\S+)', max_features=5000)

# 标签编码
mlb = MultiLabelBinarizer(classes=sorted(labels_dic.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.transform(y_val)

# 模型训练和评估
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,3), token_pattern='(\S+)', max_features=5000)),
    ('lsvc', OneVsRestClassifier(LinearSVC(C=0.1, penalty='l1', dual=False)))  # 添加了 dual=False
])
pipeline1.fit(X_train, y_train)
predicted = pipeline1.predict(X_val)

# 评估函数
def evaluation(y_val, predicted):
    # 计算f1_score，设置average='samples'以适应多标签分类
    f1_macro = f1_score(y_val, predicted, average='macro')
    f1_samples = f1_score(y_val, predicted, average='samples')
    
    # 计算hamming loss
    hamming = hamming_loss(y_val, predicted)
    
    print("F1 Score (Macro):", f1_macro)
    print("F1 Score (Samples):", f1_samples)
    print("Hamming Loss:", hamming)

evaluation(y_val, predicted)